In [2]:
import pandas as pd
import numpy as np
import json
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pickle
from tqdm import tqdm 
from ast import literal_eval
import cobra
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
from functools import partial 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

import gurobipy
gurobipy.setParam('OutputFlag', 0)
import sys
# sys.path.append('../Script/')
from common import *

# input and output

In [6]:
###input
# yeast870_path =  '../../Data/model/yeast-GEM.yml'
yeast8U_del_path = '../../Data/model/yeast8U_del_re.yml'
yeast870_path =  '../../Data/model/yeast-GEM.yml'

ess_del_reaction_df_file = '../../Results/analysis/ess_del_reaction_df_re.csv'
biolog_del_reaction_df_file = '../../Results/analysis/Biolog_del_reaction_re.csv'

cut_off = 0.3
num = 50
num_process = 50
cut_off_path = f'../../Results/not_lipid/top{num}_{cut_off}_re/'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
# rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast8_reaction_in_rxndb_json = cut_off_path + f'yeast8_reaction_in_rxndb_top{num}_{cut_off}.json'

rxndb_total_info_to_model_del_reaction_path = '../../Results/not_lipid/top50_0.3_re/rxndb_total_info_to_model_del_reaction_top50_0.3.csv'

###output
rxndb_total_info_to_model_del2_reaction_path = '../../Results/not_lipid/top50_0.3_re/rxndb_total_info_to_model_del2_reaction_top50_0.3.csv'
metabolites_info_to_GEM_del_path = '../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_del_top50_0.3.csv'
yeast8U_del2_path = '../../Data/model/Yeast-MetaTwin.yml'

In [3]:
del_reaction_lst = []

ess_del_reaction_df = pd.read_csv(ess_del_reaction_df_file,index_col=None)
ess_del_reaction_df['reaction'] = ess_del_reaction_df['reaction'].apply(lambda x : literal_eval(x))
ess_del_reaction_df = ess_del_reaction_df[ess_del_reaction_df['num']<20]

for i in ess_del_reaction_df['reaction'].to_list():
    del_reaction_lst += i

print(len(del_reaction_lst))
del_reaction_lst = list(set(del_reaction_lst))
print(len(del_reaction_lst))

print('==============================')

biolog_del_reaction_df = pd.read_csv(biolog_del_reaction_df_file,index_col=None)
biolog_del_reaction_df['reaction'] = biolog_del_reaction_df['reaction'].apply(lambda x : literal_eval(x))
biolog_del_reaction_df = biolog_del_reaction_df[biolog_del_reaction_df['num']<20]

for i in biolog_del_reaction_df['reaction'].to_list():
    del_reaction_lst += i

print(len(del_reaction_lst))
del_reaction_lst = list(set(del_reaction_lst))
print(len(del_reaction_lst))

59
43
119
79


In [6]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_del_reaction_path)
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model = rxndb_total_info_to_model[~rxndb_total_info_to_model['NO'].isin(del_reaction_lst)]
print(rxndb_total_info_to_model.shape)
rxndb_total_info_to_model.to_csv(rxndb_total_info_to_model_del2_reaction_path, index=False)

(4759, 8)
(4680, 8)


In [7]:
get_yeast8U(metabolites_info_to_GEM_path,rxndb_total_info_to_model_del2_reaction_path,yeast8_reaction_in_rxndb_json,yeast870_path,yeast8U_del2_path)

(4151, 4)
(4680, 8)
(4680, 8)
(3328, 8)


100%|██████████| 3328/3328 [00:13<00:00, 248.39it/s]


In [8]:
yeast8 = cobra.io.load_yaml_model(yeast870_path)
yeast8.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.5858,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.04144,0,0.00%
s_0925,r_1861,2.491E-06,0,0.00%
s_1277,r_1992,2.332,0,0.00%
s_1324,r_2005,0.5331,0,0.00%
s_1374,r_2020,0.0002974,0,0.00%
s_1438,r_2049,0.0003253,0,0.00%
s_1468,r_2060,0.007047,0,0.00%
s_4200,r_4593,0.0001057,0,0.00%


In [9]:
yeast8U_mod = cobra.io.load_yaml_model(yeast8U_del2_path)

In [10]:
yeast8U_mod.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654,0.6379,0,0.00%
s_0565,r_1714,1,6,100.00%
s_0796,r_1832,0.04513,0,0.00%
s_0925,r_1861,2.712E-06,0,0.00%
s_1277,r_1992,1.22,0,0.00%
s_1324,r_2005,0.04466,0,0.00%
s_1374,r_2020,0.0003239,0,0.00%
s_1438,r_2049,0.0003542,0,0.00%
s_1468,r_2060,0.007674,0,0.00%
s_4200,r_4593,0.0001151,0,0.00%


In [7]:
rxndb_to_model = pd.read_csv(rxndb_total_info_to_model_del2_reaction_path)
metabolites_info_to_GEM = pd.read_csv(metabolites_info_to_GEM_path)
met_id_lst = []
rxndb_to_model['equation_dict'] = rxndb_to_model['equation_dict'].apply(lambda x:literal_eval(x))
for i in range(len(rxndb_to_model)):
    met_id_lst += list(rxndb_to_model['equation_dict'].iloc[i].keys())

met_id_lst = list(set(met_id_lst))
print(metabolites_info_to_GEM.shape)
metabolites_info_to_GEM_del = metabolites_info_to_GEM[metabolites_info_to_GEM['ID'].isin(met_id_lst)]
print(metabolites_info_to_GEM_del.shape)
metabolites_info_to_GEM_del.to_csv(metabolites_info_to_GEM_del_path, index=False)

(4151, 4)
(4068, 4)
